In [5]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [6]:
epochs = 10
range_min = -999
range_max = 999
batch_size = 4 
num_digits = 3
num_train_samples = 100
num_test_samples = 10

In [7]:
x_train = np.random.randint(low = range_min,high = range_max+1,size = (num_train_samples,3))
x_train[:,2] = num_digits
y_train = x_train[:,0] * x_train[:,1]

In [8]:
train_data = torch.from_numpy(x_train).type(torch.FloatTensor)
train_labels = torch.from_numpy(y_train).type(torch.LongTensor)
# test_data = torch.from_numpy(x_test).type(torch.FloatTensor)
# test_labels = torch.from_numpy(y_test).type(torch.LongTensor)

In [9]:
# This model adds two numbers
class Net_add(nn.Module):
    def __init__(self):
        super(Net_add,self).__init__()
        self.fc1 = nn.Linear(2,1,bias=False)
        self.fc1.weight.data = torch.tensor([[1,1]]).type(torch.FloatTensor)
    
    def forward(self,x):
        x = self.fc1(x)
        return x
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)

model_add = torch.load('model_add')

In [10]:
# This model subtracts two numbers
class Net_sub(nn.Module):
    def __init__(self):
        super(Net_sub,self).__init__()
        self.fc1 = nn.Linear(2,1)
        self.fc1.weight.data = torch.tensor([[1,-1]]).type(torch.FloatTensor)
        self.fc1.bias.data = torch.tensor([0]).type(torch.FloatTensor)
    
    def forward(self,x):
        x = self.fc1(x)
        return x
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)
    
model_sub = torch.load('model_sub')

In [92]:
class Net_sign(nn.Module):
    def __init__(self):
        super(Net_sign,self).__init__()
        self.fc1 = nn.Linear(2,15)
        self.fc2 = nn.Linear(15,15)
        self.fc3 = nn.Linear(15,2)
    
    def forward(self,x):
        x = F.softsign(self.fc1(x))
        x = F.softsign(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
    
    def predict(self,x):
        pred = self.forward(x)
        _,out = torch.max(pred,1)
        return out.type(torch.FloatTensor)

model_initial_sign = torch.load('model_initial_sign')

In [109]:
# This model will give the sign(-1 and 1) of the product of two numbers
# input: 2 numbers
# then it uses model_initial_sign to retrieve 0 and 1
# multiplies that with 2
# then subtracts -1 to give 1 or -1 as final sign 

class Net_sign_mult(nn.Module):
    def __init__(self,model_initial_sign,model_sub):
        super(Net_sign_mult,self).__init__()
        
        self.model_initial_sign = model_initial_sign
        
        self.fc1 = nn.Linear(1,1,bias = False) # layer to multiply by 2
        self.fc1.weight.data = torch.tensor([[2]]).type(torch.FloatTensor)

        self.model_sub = model_sub

    
    def forward(self,x):
        x = self.model_initial_sign.predict(x).type(torch.FloatTensor)
        x = x.unsqueeze(-1)

        x = self.fc1(x)
        
        ones = torch.ones(x.shape)
        x = torch.cat((x,ones),1)
        x = self.model_sub.predict(x)
        return x
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)
    
model_sign_mult = torch.load('model_sign_mult')

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(2,30)
        self.fc2 = nn.Linear(30,82)
    
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x
    
    def predict(self,x):
        pred = self.forward(x)
        _,out = torch.max(pred,1)
        return out.type(torch.FloatTensor)

model_mul_0_9 = torch.load('mul_0_9')

In [53]:
# This model will give the absolute of a number
# input: 1 number
# output: absolute of that number 

class Net_abs(nn.Module):
    def __init__(self,model_add,model_sub):
        super(Net_abs,self).__init__()
        
        self.model_add = model_add
        self.model_sub = model_sub
        self.maxpool1d = nn.MaxPool1d(2)

    
    def forward(self,x):
        concat = torch.cat((x,x),1)
        add = self.model_add.predict(concat)
        sub = self.model_sub.predict(concat)
        
        concat = torch.cat((add,sub),1).unsqueeze(0)
        out = self.maxpool1d(concat).squeeze(0)
        
        concat = torch.cat((out,x),1)
        out = model_sub.predict(concat)
        
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)
    
model_abs = torch.load('model_abs')

In [15]:
# This model will separate a number into digits
# input: 2 inputs - (number,num_digits)
# output: separated digits (num_digits,)  

class Net_separate(nn.Module):
    def __init__(self,model_sub):
        super(Net_separate,self).__init__()
        
        self.divide_by_10 = nn.Linear(1,1,bias=False)
        self.divide_by_10.weight.data = torch.tensor([[0.1]]).type(torch.FloatTensor)

        self.multiply_by_10 = nn.Linear(1,1,bias=False)
        self.multiply_by_10.weight.data = torch.tensor([[10]]).type(torch.FloatTensor)
        
        self.model_sub = model_sub
    
    def forward(self,x):
        input_number = x[:,0].unsqueeze(-1)
        num_digits = x[:,1].unsqueeze(-1)
        concat = []
        data = input_number
        for i in range(num_digits[0].type(torch.IntTensor)):
            temp_data = self.divide_by_10(data).floor()
            temp = self.multiply_by_10(temp_data)
            concat_temp = torch.cat((data,temp),1)
            digit = self.model_sub.predict(concat_temp)
            
            concat.insert(0,digit)
            data = temp_data
        
        out = torch.cat(concat,1)
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)
    
model_separate = torch.load('model_separate')    

In [64]:
# This model will give the product of two absolute numbers
# input: 2 separated numbers : (2,num_digits)
# output: product 

class Net_multiply_abs(nn.Module):
    def __init__(self,model_mul_0_9,model_add,model_separate):
        super(Net_multiply_abs,self).__init__()
        
        self.model_add = model_add
        self.model_separate = model_separate
        self.model_mul_0_9 = model_mul_0_9
        
        self.final_row_add_layers = []
    
    def forward(self,x):
        n = x.shape[-1]
        m = x.shape[-1]
        
        # get initial products
        products = []
        for i in range(x.shape[-1]):
            mul = x[:,0,i:i+1]
            for j in range(x.shape[-1]):
                temp = x[:,1,j:j+1]
                concat = torch.cat((temp,mul),1)
                prod = self.model_mul_0_9.predict(concat)
                products.insert(0,prod)
                
        # get individual rows
        row_values = []
        prod_index = 0
        two = torch.from_numpy(np.repeat(np.array([2.0],ndmin = 2),x.shape[0],axis = 0)).type(torch.FloatTensor)
        for r in range(n):
            for i in range(m+1):
                if i==0:
                    temp = torch.cat((products[prod_index].unsqueeze(-1),two),1)
                    sep = self.model_separate.predict(temp)
                    carry = sep[:,0:1]
                    ones_digit = sep[:,1:]
                    row_values.append(ones_digit)
                    prod_index+=1
                    
                elif i==m:
                    row_values.append(carry)
                else:
                    concat = torch.cat((products[prod_index].unsqueeze(-1),carry),1)
                    add = self.model_add(concat)
                    concat = torch.cat((add,two),1)
                    sep = self.model_separate.predict(concat)
                    carry = sep[:,0:1]
                    ones_digit = sep[:,1:]
                    row_values.append(ones_digit)
                    prod_index+=1
        
        final_row = []
        current_ind = 0 
        for i in range(m+n):
            if i ==0:
                final_row.insert(0,row_values[0])
                temp = torch.cat((row_values[0],two),1)
                sep = self.model_separate.predict(temp)
                carry = sep[:,0:1]
                current_ind+=1
            else:
                to_add = []
                ones = []
                to_add.append(row_values[current_ind])
                ones.append(1)
                to_add.append(carry)
                ones.append(1)
                
                next_ind = current_ind + m
                while(next_ind < n*(m+1) and next_ind%(m+1)!=0):
                    to_add.append(row_values[next_ind])
                    ones.append(1)
                    next_ind+=m
                if(next_ind < n * (m+1) and next_ind%(m+1)==0):
                    to_add.append(row_values[next_ind])
                    ones.append(1)

                self.final_row_add_layers.append(nn.Linear(len(to_add),1,bias=False))
                self.final_row_add_layers[-1].weight.data = torch.tensor([ones]).type(torch.FloatTensor)
                concat = torch.cat(to_add,-1)
                add = self.final_row_add_layers[-1](concat)
                
                temp = torch.cat((add,two),1)
                sep = self.model_separate.predict(temp)
                carry = sep[:,0:1]
                ones_digit = sep[:,1:]
                final_row.insert(0,ones_digit)
                
                if (current_ind + 1) % (m+1) !=0:
                    current_ind+=1
                else:
                    current_ind+=m+1
                
                
        out = torch.cat(final_row,-1)
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)
    
model_multiply_abs = torch.load('model_multiply_abs')

In [17]:
# This model will join a list of digits to a number
# input: 1 input - (num_digits,)
# output: 1   

class Net_merge(nn.Module):
    def __init__(self):
        super(Net_merge,self).__init__()
        self.layers = []
        
    def forward(self,x):
        n = x.shape[-1]
        i = 0 
        add_list = []
        ones = []
        while(n>=1):
            temp = x[:,n-1:n]
            self.layers.append(nn.Linear(1,1,bias=False))
            self.layers[-1].weight.data = torch.tensor([[10**i]]).type(torch.FloatTensor)
            out = self.layers[-1](temp)
            i +=1
            n-=1
            add_list.append(out)
            ones.append(1)
        
        self.layers.append(nn.Linear(x.shape[-1],1,bias=False))
        self.layers[-1].weight.data = torch.tensor([ones]).type(torch.FloatTensor)
        concat = torch.cat(add_list,-1)
        out = self.layers[-1](concat)
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)

model_merge = torch.load('model_merge')

In [18]:
# this model mulltiplies a single digit number with a sign (1,-1)
# input: 2 numbers (2,)
# output: single number 
class Net_mul_sign_0_9(nn.Module):
    def __init__(self):
        super(Net_mul_sign_0_9,self).__init__()
        self.fc1 = nn.Linear(2,10)
        self.fc2 = nn.Linear(10,1)
    
    def forward(self,x):
        x = F.softsign(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def predict(self,x):
        out = self.forward(x)
        return torch.round(out).type(torch.FloatTensor)

model_mul_sign_0_9 = torch.load('model_mul_sign_0_9')

In [19]:
# this model mulltiplies a number with a sign (1,-1)
# input: 3 numbers (3,) - [number,sign,num_digits]
# output: single number 
class Net_mul_sign_abs(nn.Module):
    def __init__(self,model_separate,model_mul_sign_0_9):
        super(Net_mul_sign_abs,self).__init__()
        self.model_separate = model_separate
        self.model_mul_sign_0_9 = model_mul_sign_0_9
        self.multiply_2 = nn.Linear(1,1,bias = False)
        self.multiply_2.weight.data = torch.tensor([[2.0]])
        self.layers = []
    
    def forward(self,x):
        number = x[:,0].unsqueeze(-1)
        sign = x[:,1].unsqueeze(-1)
        num_digits = x[:,2].unsqueeze(-1)
        
        output_digits = self.multiply_2(num_digits)
        concat = torch.cat((number,output_digits),1)
        sep = self.model_separate.predict(concat)
        
        products = []
        i = 0
        j = sep.shape[-1]
        for n in range(j,0,-1):
            temp = sep[:,n-1:n]
            concat = torch.cat((temp,sign),1)
            prod = self.model_mul_sign_0_9.predict(concat)
            self.layers.append(nn.Linear(1,1,bias=False))
            self.layers[-1].weight.data = torch.tensor([[10**i]]).type(torch.FloatTensor)
            prod = self.layers[-1](prod)
            products.append(prod)
            i+=1
        
        concat = torch.cat(products,1)
        self.layers.append(nn.Linear(1,1,bias=False))
        self.layers[-1].weight.data = np.repeat(torch.tensor([[1.0]]),len(products),axis = -1)
        out = self.layers[-1](concat)
        
        return out
    
    def predict(self,x):
        out = self.forward(x)
        return out.type(torch.FloatTensor)
    
model_mul_sign_abs = torch.load('model_mul_sign_abs')

In [116]:
# this model mulltiplies two numbers
# input: 3 numbers (3,) - [number,number,num_digits]
# output: single number 
class Net_multiply(nn.Module):
    def __init__(self,model_sign_mult,model_abs,model_separate,model_multiply_abs,model_merge,model_mul_sign_abs):
        super(Net_multiply,self).__init__()
        self.model_sign_mult = model_sign_mult
        self.model_abs = model_abs
        self.model_separate = model_separate
        self.model_multiply_abs = model_multiply_abs
        self.model_merge = model_merge
        self.model_mul_sign_abs = model_mul_sign_abs
    
    def forward(self,x):
        numbers = x[:,:2]
        num_digits = x[:,2:]
        
        # get sign
        sign = self.model_sign_mult.predict(numbers)
        
        # get absolutes
        abs_inp1 = self.model_abs.predict(x[:,0:1])
        abs_inp2 = self.model_abs.predict(x[:,1:2])
        
        # separate absolutes
        concat = torch.cat((abs_inp1,num_digits),1)
        sep_1 = self.model_separate.predict(concat).unsqueeze(1)
        concat = torch.cat((abs_inp2,num_digits),1)
        sep_2 = self.model_separate.predict(concat).unsqueeze(1)
        
        # multiply absolutes
        concat = torch.cat((sep_1,sep_2),1)
        abs_prod = self.model_multiply_abs(concat)
        
        # merge absolute product
        abs_prod = self.model_merge(abs_prod)
        
        # multiply sign and absolute product
        concat = torch.cat((abs_prod,sign,num_digits),1)
        out = self.model_mul_sign_abs.predict(concat)
        
        return out
    
    def predict(self,x):
        out = self.forward(x)
        return out.type(torch.FloatTensor)
    

In [117]:
net = Net_multiply(model_sign_mult,model_abs,model_separate,model_multiply_abs,model_merge,model_mul_sign_abs)

In [118]:
print(train_data[:10])
print(train_labels[:10])
print(net.forward(train_data[:10]))

tensor([[ 373.,  568.,    3.],
        [-955.,  -97.,    3.],
        [  12.,   99.,    3.],
        [ 912.,  629.,    3.],
        [ 581., -862.,    3.],
        [-985.,  974.,    3.],
        [-454., -530.,    3.],
        [ 152.,   47.,    3.],
        [-297.,  -72.,    3.],
        [-221.,  629.,    3.]])
tensor([ 211864,   92635,    1188,  573648, -500822, -959390,  240620,    7144,
          21384, -139009])


C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


tensor([[ 211864.],
        [  92635.],
        [   1188.],
        [ 573648.],
        [-500822.],
        [-959390.],
        [ 240620.],
        [   7144.],
        [  21384.],
        [-139009.]], grad_fn=<MmBackward>)


In [119]:
torch.save(net,'model_multiply')

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_multiply. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_sign_mult. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_sign. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_sub. It won't be checked for correctness upon loading.
